# MIS High level

In [1]:
import networkx as nx

# Create a simple triangle graph
graph = nx.Graph()
graph.add_edges_from([(0, 1), (0, 2)])

In [2]:

from mis.shared.types import MISInstance
from mis.pipeline.config import SolverConfig
from mis import MISSolver

Solve using the configuration without quantum

In [3]:

# Define classical solver configuration
from mis.shared.types import MethodType


config = SolverConfig(
    method = MethodType.EAGER,
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)


# Run the solver
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].energy)

MIS solution: [1, 2]
Solution cost: 0


Solve using the configuration with QTip

In [4]:
from mis.pipeline.backends import QutipBackend


config = SolverConfig(
    method = MethodType.EAGER,
    backend = QutipBackend(),
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)


# Run the solver
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].energy)

MIS solution: [1, 2]
Solution cost: -0.5697872454017736


Solve using Remote QPU backend 

In [ ]:
from mis.pipeline.backends import RemoteQPUBackend

USERNAME="username"
PROJECT_ID="123"
PASSWORD=None

if PASSWORD is not None:
    config = SolverConfig(
        method = MethodType.EAGER,
        backend = RemoteQPUBackend(
            username=USERNAME,
            project_id=PROJECT_ID,
            password=PASSWORD
        ),
        max_iterations=1
    )

    # Create the MIS instance
    instance = MISInstance(graph)

    # Run the solver
    solver = MISSolver(instance, config)
    solutions = solver.solve().result()

    # Display results
    print("MIS solution:", solutions[0].nodes)
    print("Solution cost:", solutions[0].energy)

ImportError: cannot import name 'Connecter' from 'mis' (/home/david/Documents/Code/maximum-independent-set/mis/__init__.py)

In [ ]:
conn = Connecter(backend=BackendType.REMOTE_EMUMPS,  # update the backend
    project_id="project-123",
    username="your_username",
    password="your_password")

conn = None

# integrate limitations for using backends with different types of graphs 
# 15 atoms - QTips
# Add warnings about size about upper bounds - state vector emulation (15 atoms)
# emu mps ??

In [ ]:
# Point - keep the connection separate than solver configration 
# Keep the connector as an optional argument somewhere below

# Set up the quantum solver config
config = SolverConfig(
    use_quantum=True,
    method = "wmis",  # or "greedy"
    max_iterations=3,
    connector = conn
)

# Solve the problem using quantum solver
solver = MISSolver(instance, config)

# raise errors here when the register are not compatible 
result = solver.solve()

In [ ]:
# Print output
print("Quantum MIS solution:", result.bitstrings)
print("Solution cost:", result.costs)